<a href="https://colab.research.google.com/github/FREYDER18/SYS/blob/main/PUNTOOO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***SOLUCION PUNTO 2 "PROGRAMA"***

In [ ]:
%%writefile 2_Punto_2.py
# ================================
# 📦 Importación de librerías
# ================================
import streamlit as st  # Para crear el dashboard interactivo
import numpy as np  # Operaciones numéricas y manejo de arreglos
from scipy.fft import fft, ifft, fftshift, ifftshift, fftfreq  # Herramientas para la transformada de Fourier
from scipy.io import wavfile  # Para leer archivos WAV
import matplotlib.pyplot as plt  # Para graficar señales
import yt_dlp  # Para descargar audio desde YouTube
import os  # Manejo de archivos
import subprocess  # Para ejecutar comandos externos como ffmpeg

# ================================
# 🎨 Configuración visual del Dashboard
# ================================
st.set_page_config(
    page_title="🎛️ Modulador SSB-AM (FFT)",  # Título en la pestaña del navegador
    page_icon="🎧",  # Ícono de la app
    layout="wide",  # Distribución ancha
    initial_sidebar_state="expanded"  # Mostrar la barra lateral al iniciar
)

# Estilo CSS personalizado para colores y estructura visual del dashboard
st.markdown("""
<style>
body {
    background-color: #e8f0fe;  /* Fondo claro del cuerpo */
    color: #1a1a1a;  /* Color de texto principal */
    font-family: 'Arial', sans-serif;  /* Fuente moderna */
}

h1, h2, h3 {
    color: #003366;  /* Colores oscuros para los títulos */
}

.sidebar .sidebar-content {
    background-color: #d6e4f0;  /* Color de fondo de la barra lateral */
    color: #1a1a1a;  /* Color del texto lateral */
}

.reportview-container .markdown-text-container {
    font-size: 16px;  /* Tamaño del texto */
}

.block-container {
    padding: 1rem 2rem;  /* Relleno del contenedor principal */
    background-color: #ffffff;  /* Fondo blanco para el contenido */
    border-radius: 10px;  /* Bordes redondeados */
    box-shadow: 0 0 10px rgba(0,0,0,0.05);  /* Sombra ligera */
}
</style>
""", unsafe_allow_html=True)

# ================================
# 🧾 Título y descripción general
# ================================
st.title("🎧 Dashboard de Modulación SSB-AM por FFT")  # Título principal del dashboard

# Descripción general
st.write(
    "Este panel permite simular y visualizar el proceso de modulación y demodulación \
    de señales de banda lateral única (SSB) mediante la Transformada Rápida de Fourier."
)

# Información útil para el usuario
st.info(
    "💡 Para usar audio desde YouTube necesitas tener `yt-dlp` y `ffmpeg` instalados en el entorno.",
    icon="📌"
)

# ================================
# ⚙️ Funciones auxiliares
# ================================
@st.cache_data  # Guarda en caché los resultados de esta función para no volver a ejecutarla

def load_audio_from_youtube(url, duration_s=5, start_s=0):
    """
    Descarga un fragmento de audio desde una URL de YouTube,
    lo convierte a formato WAV mono, lo normaliza y lo carga como vector de señal.
    """
    temp_wav_file = 'temp_audio.wav'  # Archivo WAV temporal original
    cropped_wav_file = 'cropped_audio.wav'  # Archivo WAV recortado a la duración deseada
    try:
        # Configuración del descargador
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
            'outtmpl': 'temp_audio',
            'quiet': True,
        }
        # Elimina archivos anteriores si existen
        for f in [temp_wav_file, cropped_wav_file]:
            if os.path.exists(f): os.remove(f)

        # Descarga del audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Recorte del audio descargado con ffmpeg
        subprocess.run([
            'ffmpeg', '-i', temp_wav_file, '-ss', str(start_s),
            '-t', str(duration_s), '-acodec', 'pcm_s16le',
            '-ar', '44100', '-ac', '1', cropped_wav_file, '-y'
        ], check=True, capture_output=True)

        # Cargar archivo WAV recortado
        fs_audio, audio_data = wavfile.read(cropped_wav_file)
        if audio_data.ndim > 1:
            audio_data = audio_data.mean(axis=1)  # Si es estéreo, lo convierte a mono
        audio_data = audio_data / np.max(np.abs(audio_data))  # Normaliza la amplitud
        m_t = audio_data  # Señal mensaje
        t = np.linspace(start_s, start_s + len(m_t)/fs_audio, len(m_t), endpoint=False)  # Vector de tiempo

        # Limpieza de archivos temporales
        for f in [temp_wav_file, cropped_wav_file]:
            if os.path.exists(f): os.remove(f)

        return m_t, fs_audio, t

    except Exception as e:
        st.error(f"❌ Error al procesar el audio: {e}")
        for f in [temp_wav_file, cropped_wav_file]:
            if os.path.exists(f): os.remove(f)
        return None, None, None

# ------------------------
# Visualización en tiempo
# ------------------------
def plot_signal_time(t, sig, title):
    fig, ax = plt.subplots(figsize=(10, 4))  # Crea la figura
    ax.plot(t, sig, lw=1)  # Dibuja la señal
    ax.set_title(title)  # Título
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)  # Cuadrícula
    plt.tight_layout()  # Ajuste automático
    return fig

# ------------------------
# Visualización en frecuencia
# ------------------------
def plot_signal_freq(sig, fs, title, xlim_freq=None):
    fig, ax = plt.subplots(figsize=(10, 4))
    N = len(sig)
    if N == 0: return fig  # Previene errores si la señal está vacía
    yf = fftshift(fft(sig))  # FFT y centrado
    xf = fftshift(fftfreq(N, 1 / fs))  # Frecuencias
    ax.plot(xf, np.abs(yf) / N, lw=1)  # Magnitud normalizada
    ax.set_title(title)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_ylabel("Magnitud")
    ax.grid(True)
    if xlim_freq:
        ax.set_xlim(xlim_freq)  # Límite de frecuencia si se especifica
    plt.tight_layout()
    return fig

# ------------------------
# Modulación SSB (FFT)
# ------------------------
def modulate_ssb_fft(m_t, t, fs, fc, sideband_type):
    s_dsb_t = m_t * (2 * np.cos(2 * np.pi * fc * t))  # Modulador DSB
    S_dsb_f = fftshift(fft(s_dsb_t))  # FFT centrada
    freqs = fftshift(fftfreq(len(t), 1 / fs))  # Vector de frecuencias
    if sideband_type == 'USB (Superior)':
        mask = np.where(freqs > fc, 1, 0)  # Filtro banda superior
    else:
        mask = np.where(freqs < -fc, 1, 0)  # Filtro banda inferior
    S_ssb_f = S_dsb_f * mask  # Aplicar el filtro
    s_ssb_t = np.real(ifft(ifftshift(S_ssb_f)))  # Señal modulada SSB
    return s_ssb_t

# ------------------------
# Demodulación SSB (FFT)
# ------------------------
def demodulate_ssb_fft(s_ssb_t, t, fs, fc, message_bw):
    v_t = s_ssb_t * (2 * np.cos(2 * np.pi * fc * t))  # Multiplicación por portadora
    V_f = fftshift(fft(v_t))  # FFT centrada
    freqs = fftshift(fftfreq(len(t), 1 / fs))
    cutoff_freq = message_bw * 1.2  # Corte del filtro paso bajo
    lpf_mask = np.where(np.abs(freqs) <= cutoff_freq, 1, 0)  # Filtro LPF
    M_demod_f = V_f * lpf_mask  # Aplicar filtro
    m_demod_t = np.real(ifft(ifftshift(M_demod_f)))  # Señal demodulada
    return m_demod_t


In [ ]:
!mv 2_Punto_2.py pages/

##***Inicialización del Dashboard a partir de túnel local***


In [ ]:
# 🔽 Descargar Cloudflared (herramienta de túnel de Cloudflare)
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64

# 🔧 Dar permisos de ejecución al binario descargado
!chmod +x cloudflared-linux-amd64

# 📁 Mover el binario a una carpeta del sistema para poder usarlo como comando
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 🚀 Iniciar la aplicación de Streamlit
# ❗ Reemplaza '2_Punto_2.py' por el nombre de tu archivo principal si es diferente
# ⏺ Redirige toda la salida al archivo logs.txt y corre en segundo plano
!streamlit run 2_Punto_2.py &>/content/logs.txt &

# 🌐 Crear un túnel seguro desde Cloudflare al puerto 8501 (por defecto en Streamlit)
# ⏺ Guarda la salida en cloudflared.log y se ejecuta en segundo plano
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# ⏳ Esperar unos segundos a que el túnel se cree
import time
time.sleep(5)  # Da tiempo a que se genere la URL pública

# 🧠 Usar expresiones regulares para extraer la URL pública desde el log
import re
found_context = False  # Bandera para indicar si ya se encontró el inicio del contexto relevante

# 📖 Leer el archivo de log generado por cloudflared
with open('/content/cloudflared.log') as f:
    for line in f:
        # 🔍 Detecta el punto donde se creó el túnel
        if "Your quick Tunnel has been created" in line:
            found_context = True

        # 🌐 Buscar una URL válida en las siguientes líneas
        if found_context:
            match = re.search(r'https?://\S+', line)  # Buscar enlaces http o https
            if match:
                url = match.group(0)  # Extraer la URL detectada
                print(f'✅ Tu aplicación está disponible en: {url}')  # Mostrar al usuario
                break  # Terminar el ciclo cuando se encuentra la URL


#***Finalización de ejecución del Dashboard***


In [ ]:
import os

# 🧾 Mostrar menú al usuario
print("\n🔴 FINALIZACIÓN DEL DASHBOARD")
print("¿Deseas cerrar la aplicación de Streamlit?")
print("1 - Sí, finalizar")
print("2 - No, mantener en ejecución\n")

# 📥 Captura la entrada del usuario
res = input("👉 Digite una opción (1 o 2): ").strip()

# ✅ Opción para finalizar
if res == "1":
    try:
        # 🛑 Finalizar procesos de Streamlit (funciona en Linux)
        os.system("pkill streamlit")
        print("\n✅ El proceso de Streamlit ha sido finalizado exitosamente.")
    except Exception as e:
        print(f"❌ Ocurrió un error al intentar cerrar Streamlit: {e}")
else:
    print("\nℹ️ El dashboard seguirá ejecutándose.")



🔴 FINALIZACIÓN DEL DASHBOARD
¿Deseas cerrar la aplicación de Streamlit?
1 - Sí, finalizar
2 - No, mantener en ejecución

